# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Loading the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [14]:
# Encoding "homeowner" with LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])

# Split the Data into Training and Testing

In [15]:
# Creating our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)

# Creating our target
y = df['loan_status']

In [16]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [17]:
# Checking the balance of our target values
df['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [18]:
# Creating X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [19]:
# Creating the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [20]:
# Fitting the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [21]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [23]:
# Calculating the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9520479254722232

In [24]:
# Displaying the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  563,    56],
       [  102, 18663]])

In [25]:
# Printing the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.91      0.99      0.88      0.95      0.90       619
   low_risk       1.00      0.99      0.91      1.00      0.95      0.91     18765

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



# Oversampling

In this section, I will compare two oversampling algorithms to determine which algorithm results in the best performance. I will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, I am completing the folliowing steps:

1. Viewing the count of the target classes using `Counter` from the collections library. 
3. Using the resampled data to train a logistic regression model.
3. Calculating the balanced accuracy score from sklearn.metrics.
4. Printing the confusion matrix from sklearn.metrics.
5. Generating a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: I used a random state of 1 for each sampling algorithm to ensure consistency between tests.

### Naive Random Oversampling

In [26]:
# Resampling the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled_1, y_resampled_1 = ros.fit_resample(X_train, y_train)

# Viewing the count of target classes with Counter
Counter(y_resampled_1)


Counter({'low_risk': 56271, 'high_risk': 56271})

In [27]:
# Training the Logistic Regression model using the resampled data
model_1 = LogisticRegression(solver='lbfgs', random_state=1)
model_1.fit(X_resampled_1, y_resampled_1)

LogisticRegression(random_state=1)

In [28]:
# Calculating the balanced accuracy score
y_predicted_1 = model_1.predict(X_test)
balanced_accuracy_score(y_test, y_predicted_1)

0.9936781215845847

In [29]:
# Displaying the confusion matrix
confusion_matrix(y_test, y_predicted_1)

array([[  615,     4],
       [  116, 18649]])

In [30]:
# Printing the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predicted_1, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8413    0.9935    0.9938    0.9111    0.9937    0.9874       619
   low_risk     0.9998    0.9938    0.9935    0.9968    0.9937    0.9874     18765

avg / total     0.9947    0.9938    0.9935    0.9941    0.9937    0.9874     19384



### SMOTE Oversampling

In [31]:
# Resampling the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled_2, y_resampled_2 = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train, y_train)

# Viewing the count of target classes with Counter
Counter(y_resampled_2)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [32]:
# Training the Logistic Regression model using the resampled data
model_2 = LogisticRegression(solver='lbfgs', random_state=1)
model_2.fit(X_resampled_2, y_resampled_2)

LogisticRegression(random_state=1)

In [33]:
# Calculating the balanced accuracy score
y_predicted_2 = model_2.predict(X_test)
balanced_accuracy_score(y_test, y_predicted_2)

0.9936781215845847

In [34]:
# Displaying the confusion matrix
confusion_matrix(y_test, y_predicted_2)

array([[  615,     4],
       [  116, 18649]])

In [35]:
# Printing the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predicted_2, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8413    0.9935    0.9938    0.9111    0.9937    0.9874       619
   low_risk     0.9998    0.9938    0.9935    0.9968    0.9937    0.9874     18765

avg / total     0.9947    0.9938    0.9935    0.9941    0.9937    0.9874     19384



# Undersampling

In this section, I will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. I will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. Viewing the count of the target classes using `Counter` from the collections library. 
3. Using the resampled data to train a logistic regression model.
3. Calculating the balanced accuracy score from sklearn.metrics.
4. Displaying the confusion matrix from sklearn.metrics.
5. Generating a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: I used a random state of 1 for each sampling algorithm to ensure consistency between tests

In [36]:
# Resampling the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled_3, y_resampled_3 = cc.fit_resample(X_train, y_train)

# Viewing the count of target classes with Counter
Counter(y_resampled_3)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [37]:
# Training the Logistic Regression model using the resampled data
model_3 = LogisticRegression(solver='lbfgs', random_state=1)
model_3.fit(X_resampled_3, y_resampled_3)

LogisticRegression(random_state=1)

In [39]:
# Calculating the balanced accuracy score
y_predicted_3 = model_3.predict(X_test)
balanced_accuracy_score(y_test, y_predicted_3)

0.9865149130022852

In [40]:
# Displaying the confusion matrix
confusion_matrix(y_test, y_predicted_3)

array([[  606,    13],
       [  112, 18653]])

In [41]:
# Printing the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predicted_3, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8440    0.9790    0.9940    0.9065    0.9865    0.9717       619
   low_risk     0.9993    0.9940    0.9790    0.9967    0.9865    0.9746     18765

avg / total     0.9943    0.9936    0.9795    0.9938    0.9865    0.9745     19384



# Combination (Over and Under) Sampling

In this section, I will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. I will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. Viewing the count of the target classes using `Counter` from the collections library. 
3. Using the resampled data to train a logistic regression model.
3. Calculating the balanced accuracy score from sklearn.metrics.
4. Displaying the confusion matrix from sklearn.metrics.
5. Generating a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: I used a random state of 1 for each sampling algorithm to ensure consistency between tests

In [51]:
# Resampling the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled_4, y_resampled_4 = sm.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_4)

Counter({'high_risk': 55609, 'low_risk': 55948})

In [52]:
# Training the Logistic Regression model using the resampled data
model_4 = LogisticRegression(solver='lbfgs', random_state=1)
model_4.fit(X_resampled_4, y_resampled_4)

LogisticRegression(random_state=1)

In [53]:
# Calculating the balanced accuracy score
y_predicted_4 = model_4.predict(X_test)
balanced_accuracy_score(y_test, y_predicted_4)

0.9935182494822666

In [54]:
# Displaying the confusion matrix
confusion_matrix(y_test, y_predicted_4)

array([[  615,     4],
       [  122, 18643]])

In [55]:
# Printing the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predicted_4, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8345    0.9935    0.9935    0.9071    0.9935    0.9871       619
   low_risk     0.9998    0.9935    0.9935    0.9966    0.9935    0.9871     18765

avg / total     0.9945    0.9935    0.9935    0.9938    0.9935    0.9871     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   The Naive Random Oversampling and SMOTE Oversampling have the best balanced accuracy scores. 

2. Which model had the best recall score?

    The Naive Random Oversampling and SMOTE Oversampling also have the best recall scores (0.9938).

3. Which model had the best geometric mean score?

    The Naive Random Oversampling and SMOTE Oversampling have the best geometric mean scores (0.9937). 
